In [0]:
# DO NOT EDIT/ DELETE THIS BLOCK;
#


In [1]:
# we have to load drive coz the models & dataset are there 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab or local 
%cd /content/drive/My\ Drive/SNLP\ Project
# %cd Source/repos/Sentiment-Analysis-using-Deep-Learning

/content/drive/My Drive/SNLP Project


In [3]:
# list content of drive - verify you are where you are supposed to be
%ls

 amz_all_beauty/                    meta_elec_small_lstm.tsv
 amz_all_electronics/               meta_lstm_small_200dim.tsv
 CleantData_Apr-03-2020_01-31.zip   ModelResults/
 dataset_dumps.json                 ModelVisualization/
 Electronics_5.json.gz             'Project Ideas.gdoc'
 FinalModelResults/                'Project Proposal.gdoc'
'Final Report'/                    'Report MetaData'/
 glove.6B.200d.txt                 'Sentiment Analysis - Deep Learning.pptx'
 glove.6B.zip                       vecs_elec_small_lstm.tsv
 Lexical_analysis/                  vecs_lstm_small_200dim.tsv


In [0]:
# load your choice of dataset here . Specify paths as folder_datestring/file_datestring.zip

data_corpus = {
    'elec_small_lstm' : {
        'model_path' : 'ModelResults/v3_767c02af219c4133946edcaf89e67387/model.h5',
        'train_data_path' : 'amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Train_2000_Apr-03-2020_06-46.zip',
        'test_data_path' : 'amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Test_2000_Apr-03-2020_06-46.zip',
    },
    'elec_med_lstm' : {
        'model_path' : 'ModelResults/v3_lstm_1f9ff1c3924d4bac8894dda0fc3bf5bf/model.h5',
        'train_data_path' : 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Train_20000_Apr-03-2020_06-52.zip',
        'test_data_path' : 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Test_20000_Apr-03-2020_06-52.zip',
    },
    'elec_med_bilstm_glove' : {
        'model_path' : 'ModelResults/v3_bilstm_glove_739665f6a8794e98b874bec09723d536/model.h5',
        'train_data_path' : 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Train_20000_Apr-03-2020_06-52.zip',
        'test_data_path' : 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Test_20000_Apr-03-2020_06-52.zip',
    },
    'elec_med_cnn_gru' : {
        'model_path' : 'ModelResults/v3_cnn_gru_2_5123azxc1abe9af33fd580bf998285182/model.h5',
        'train_data_path' : 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Train_20000_Apr-03-2020_06-52.zip',
        'test_data_path' : 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Test_20000_Apr-03-2020_06-52.zip',
    },    
}


# GLOBAL VARIABLE -> which model u want to load 
CHOSEN_MODEL_TO_LOAD = 'elec_med_bilstm_glove'



In [46]:
Train_ZipCSVFileName = data_corpus[CHOSEN_MODEL_TO_LOAD]['train_data_path']
Test_ZipCSVFileName = data_corpus[CHOSEN_MODEL_TO_LOAD]['test_data_path']

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         100000 non-null  float64
 1   reviewTime      100000 non-null  object 
 2   reviewerID      100000 non-null  object 
 3   asin            100000 non-null  object 
 4   reviewText      100000 non-null  object 
 5   summary         100000 non-null  object 
 6   unixReviewTime  100000 non-null  int64  
 7   reviewText_len  100000 non-null  int64  
 8   summary_len     100000 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 6.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         10000 non-null  float64
 1   reviewTime      10000 non-null  object 
 2   reviewerID      10000 non-null 

In [47]:

# Get names of indexes for which column Age has value 30
index_neutrals_train = df_train[ df_train['overall'] == 3 ].index
index_neutrals_test = df_test[ df_test['overall'] == 3 ].index
 
# Delete these row indexes from dataFrame
df_train.drop(index_neutrals_train , inplace=True)
df_test.drop(index_neutrals_test , inplace=True)

df_train.loc[(df_train.overall == 1),'overall']= 1
df_train.loc[(df_train.overall == 2),'overall']= 1
df_train.loc[(df_train.overall == 4),'overall']= 5
df_train.loc[(df_train.overall == 5),'overall']= 5

df_test.loc[(df_test.overall == 1),'overall']= 1
df_test.loc[(df_test.overall == 2),'overall']= 1
df_test.loc[(df_test.overall == 4),'overall']= 5
df_test.loc[(df_test.overall == 5),'overall']= 5

df_train['reviewText_len'].describe()
# Since the mean average review size is around 145 chars and max is 400, I can safely set the max [summary + review] Text Limit to 400 
	

count    80000.000000
mean       148.534925
std        109.884928
min          1.000000
25%         49.000000
50%        129.000000
75%        228.000000
max        399.000000
Name: reviewText_len, dtype: float64

In [48]:
# Tokenize the data
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each segment.
MAX_SEQUENCE_LENGTH = 250

# Reference https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, lower=True)
tokenizer.fit_on_texts(df_train['summary'] + ' DELIM '+ df_train['reviewText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 26904 unique tokens.


In [0]:
# we have the word dict now lets load the trained model 
from tensorflow.keras.models import load_model
# Load the model
model = load_model(data_corpus[CHOSEN_MODEL_TO_LOAD]['model_path'])



In [52]:
# TEST YOUR REVIEW HERE 
# sample reviews if needed https://www.amazon.com/s?k=smartphones

#user_demo_summary = "Great samsung s10"
#user_demo_review  = "I had a great experience buying this phone! I have been using it so long now, the battery is a bit bad but overall it works fine. The touch screen is immaculate"


user_demo_summary = "2 year old samsung s10"
user_demo_review  = "I had a decent experience since I bought this phone. I have been using it so long now, the battery is not too good but overall it works fine. The touch screen is sort of working"




seq = tokenizer.texts_to_sequences([user_demo_summary+ ' DELIM '+ user_demo_review])
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)


print(pred)
if np.argmax(pred) == 0:
  print('Overall Sentiment Negative :(')
else:
  print('Overall Sentiment Positive :)')
  

[[0.9468437  0.05476103]]
Overall Sentiment Negative :(
